# Общие эпитопы ковида и человека
### Ищем регионы шириной 9 и 14 а.к., сходство минимум 90%
***

# [18.09]

In [3]:
peplen = 5

In [19]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

batch = 20
peps = []
with open("wuhan_hu_1.faa", "r") as handle:
    q = 1
    for record in SeqIO.parse(handle, "fasta"):
        for i in range(1, len(record.seq)-peplen):
            seq_name = record.id + "_" + str(i)
            seq = Seq(str(record.seq[i:i+peplen]))
            sr = SeqRecord(seq, seq_name, '', '')
            peps.append(sr)
            if(len(peps)>batch):
                fname="peptides/batch_"+str(q)+".fasta"
                with open(fname, "w") as output_handle:
                    SeqIO.write(peps, output_handle, "fasta")
                    q = q + 1
                    peps = []

In [ ]:
!blastp -db uniprot_human -query peptides/batch_1.fasta -word_size 2 -evalue 1000 -outfmt '6 std nident salltitles'

In [30]:
!mkdir -p result
!cat blast/*.out > result/blast.out 

In [27]:
print(peplen)
import pandas as pd
blast = pd.read_csv("result/blast.out", sep="\t", header = None)
blast.columns=["qseqid", 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore', 'nident']
blast_ok = blast[blast["nident"]==peplen]
blast_ok
    

5


,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,nident
1,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|B3KUL3|B3KUL3_HUMAN,100.0,5,0,0,1,5,85,89,66.0,17.7,5
2,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|F6VJS9|F6VJS9_HUMAN,100.0,5,0,0,1,5,9,13,68.0,17.7,5
3,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|A0A087WZ62|A0A087WZ62_HUMAN,100.0,5,0,0,1,5,174,178,96.0,16.9,5
6,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|A0A087WVS3|A0A087WVS3_HUMAN,100.0,5,0,0,1,5,370,374,31.0,18.5,5
7,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|A0A087WVW4|A0A087WVW4_HUMAN,100.0,5,0,0,1,5,368,372,31.0,18.5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1486,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|A0A126GWB3|A0A126GWB3_HUMAN,100.0,5,0,0,1,5,170,174,39.0,18.1,5
1487,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|A0A286YFF0|A0A286YFF0_HUMAN,100.0,5,0,0,1,5,170,174,49.0,18.1,5
1488,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|A0A087X1K4|A0A087X1K4_HUMAN,100.0,5,0,0,1,5,176,180,56.0,17.7,5
1490,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|I3L466|I3L466_HUMAN,100.0,5,0,0,1,5,851,855,85.0,17.3,5


# [16.09]

In [3]:
gunzip sequences_2020-06-16_13-19.fasta.gz

In [6]:
makeblastdb -in sequences_2020-06-16_13-19.fasta -out uniprot_human -dbtype prot



Building a new DB, current time: 09/16/2020 16:03:22
New DB name:   /data11/bio/PROJECTS/CoV/commotSeqsWithHuman/uniprot_human
New DB title:  sequences_2020-06-16_13-19.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 54715 sequences in 1.02822 seconds.


In [5]:
cp ../metatranscriptome/ref/wuhan_hu_1.gb .

In [2]:
#faa_sars="../gisaid/gisaid/prokka/hCoV_19_Wuhan_Hu_1/hCoV_19_Wuhan_Hu_1.faa"

In [8]:
GB2faa.pl wuhan_hu_1.gb > wuhan_hu_1.faa
faa_sars="wuhan_hu_1.faa"

In [15]:
blastp -db uniprot_human -query $faa_sars -outfmt 6 -evalue 30 > blast_results
wc -l blast_results

143 blast_results


In [10]:
blastp -db uniprot_human -query $faa_sars > blast_results_full

### look blast results
***

In [4]:
blast = read.delim("blast_results", head = F)

In [6]:
colnames(blast) = c("qid", "sid", "identity", "length", "mismatches", "gaps", "qstart", "qend", "sstart", "send", "evalue", "bitscore")

In [10]:
blast = blast[order(-blast$identity),]


In [11]:
head(blast)

,qid,sid,identity,length,mismatches,gaps,qstart,qend,sstart,send,evalue,bitscore
14,hCoV_19_Wuhan_Hu_1_00002,tr|A0A5K1VW56|A0A5K1VW56_HUMAN,53.846,26,12,0,1096,1121,237,262,3.2,32.7
44,hCoV_19_Wuhan_Hu_1_00004,tr|A0A0A0MT16|A0A0A0MT16_HUMAN,50.000,20,10,0,127,146,12,31,3.4,29.6
50,hCoV_19_Wuhan_Hu_1_00007,tr|K7EQ58|K7EQ58_HUMAN,47.368,19,10,0,35,53,116,134,3.7,26.6
51,hCoV_19_Wuhan_Hu_1_00007,tr|E9PL27|E9PL27_HUMAN,47.368,19,10,0,35,53,116,134,3.8,26.6
52,hCoV_19_Wuhan_Hu_1_00007,tr|E9PRR6|E9PRR6_HUMAN,47.368,19,10,0,35,53,116,134,4.5,26.6
57,hCoV_19_Wuhan_Hu_1_00008,tr|B4DWB8|B4DWB8_HUMAN,44.828,29,15,1,76,103,140,168,2.3,27.7
